<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing

In this workflow within PyCCAPT, we can crope the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.tutorials.tutorials_helpers import helper_calibration
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helpers import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helpers import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_selection
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_list

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, please install the pytables library by running the cell below. Befor running the cell, uncomment the cell by removing the # sign at the begining of the cell.
</div>


In [2]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [2]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping Workflow
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [3]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, max_mc)

Dropdown(description='Data mode:', options=('pyccapt', 'leap_epos', 'leap_pos', 'ato_v6'), value='pyccapt')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [4]:
variables = helper_data_loader.load_data(dataset_path, max_mc, flight_path_length, pulse_mode, tdc)

The maximum possible TOF is: 5010 ns
The data will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/data_processing/
The dataset name after saving is: data_1642_Aug-30-2023_16-05_Al_test4
The figures will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/data_processing/
{'apt': ['high_voltage', 'hour', 'main_chamber_vacuum', 'minute', 'num_events', 'pulse', 'second', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data'], 'time': []}
The number of data over max_tof: 242
Total number of Ions: 12312754


In [5]:
#load data, if it exists,
try:
    variables.data = data_tools.load_data(variables.result_data_path + '//' + variables.result_data_name + '.h5', tdc='pyccapt', mode='processed')
    variables.range_data = data_tools.read_hdf5_through_pandas(variables.result_data_path + '/' + 'range_' + variables.dataset_name+ '.h5')
    print('Continue from the point based on the loaded data')
    # exctract needed data from Pandas data frame as an numpy array
    data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
    display(variables.data)
    display(variables.range_data)
except ValueError as e:
    Print('There was no data available')
    print(e)

Continue from the point based on the loaded data
The maximum time of flight: 5010


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,16.241534,20.424568,5.380027,27.055355,29.321677,5014.109863,1002.822021,16975,571.039847,626.025696,2.053878,2.582857,0,0
1,0.649997,12.133287,1.128566,27.053985,29.829835,5014.109863,1002.822021,17011,571.026315,611.102661,0.078367,1.462857,36,2
2,8.955535,-2.694827,0.666238,27.363812,29.768729,5014.109863,1002.822021,17458,574.079711,608.448608,1.074286,-0.323265,447,1
3,-2.984976,2.409889,0.111661,27.333617,30.148581,5014.109863,1002.822021,17993,573.782894,609.607605,-0.355918,0.287347,535,1
4,9.509804,14.745539,2.376025,27.057828,29.561435,5014.109863,1002.822021,18541,571.064287,614.175049,1.162449,1.802449,548,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10359555,21.613046,4.708798,57.016720,27.216647,29.476320,6356.779785,1271.355957,3630,572.631530,548.873169,2.083265,0.453878,279,1
10359556,-10.062750,0.069160,54.646308,27.119814,30.066950,6356.779785,1271.355957,4153,571.676523,546.424866,-0.950204,0.006531,523,1
10359557,34.298491,1.530892,61.409855,26.956003,28.806147,6356.779785,1271.355957,4266,570.057059,557.507385,3.438367,0.153469,113,1
10359558,27.449792,6.196191,58.914668,26.993966,29.074990,6356.779785,1271.355957,4286,570.432804,551.534058,2.690612,0.607347,20,1


,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.01,1.002519,0.847359,1.166235,#b2aa2d,[H],[1],[1],1.0
1,${}^{27}Al^{2+}$,13.49,13.433556,13.241603,14.382197,#e7e0d1,[Al],[1],[27],2.0
2,${}^{27}Al^{+}$,26.98,26.967346,26.223492,28.970731,#e7e0d1,[Al],[1],[27],1.0


In [6]:
variables.data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,16.241534,20.424568,5.380027,27.055355,29.321677,5014.109863,1002.822021,16975,571.039847,626.025696,2.053878,2.582857,0,0
1,0.649997,12.133287,1.128566,27.053985,29.829835,5014.109863,1002.822021,17011,571.026315,611.102661,0.078367,1.462857,36,2
2,8.955535,-2.694827,0.666238,27.363812,29.768729,5014.109863,1002.822021,17458,574.079711,608.448608,1.074286,-0.323265,447,1
3,-2.984976,2.409889,0.111661,27.333617,30.148581,5014.109863,1002.822021,17993,573.782894,609.607605,-0.355918,0.287347,535,1
4,9.509804,14.745539,2.376025,27.057828,29.561435,5014.109863,1002.822021,18541,571.064287,614.175049,1.162449,1.802449,548,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10359555,21.613046,4.708798,57.016720,27.216647,29.476320,6356.779785,1271.355957,3630,572.631530,548.873169,2.083265,0.453878,279,1
10359556,-10.062750,0.069160,54.646308,27.119814,30.066950,6356.779785,1271.355957,4153,571.676523,546.424866,-0.950204,0.006531,523,1
10359557,34.298491,1.530892,61.409855,26.956003,28.806147,6356.779785,1271.355957,4266,570.057059,557.507385,3.438367,0.153469,113,1
10359558,27.449792,6.196191,58.914668,26.993966,29.074990,6356.779785,1271.355957,4286,570.432804,551.534058,2.690612,0.607347,20,1


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [ ]:
helper_temporal_crop.call_plot_crop_experiment(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spacial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [ ]:
helper_special_crop.call_plot_crop_fdm(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [ ]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data)

# add two calculated array to the croped dataset
variables.data['pulse_pi'] = pulse_pi.astype(np.uintc)
variables.data['ion_pp'] = ion_pp.astype(np.uintc)

# exctract needed data from Pandas data frame as an numpy array
variables.dld_high_voltage = variables.data['high_voltage (V)'].to_numpy()
variables.dld_pulse = variables.data['pulse'].to_numpy()
variables.dld_t = variables.data['t (ns)'].to_numpy()
variables.dld_x_det = variables.data['x_det (cm)'].to_numpy()
variables.dld_y_det = variables.data['y_det (cm)'].to_numpy()

# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data)) * 100)))
#percentage of double event per pulse
print('percentage of double event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)))

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [ ]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [ ]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False)
display(variables.data)
display(variables.data.dtypes)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight Calibration Workflow

</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
calibration_mode = widgets.Dropdown(
    options=[('time_of_flight', 'tof'), ('mass_to_charge', 'mc')],
    description='calibration mode:')
display(calibration_mode)

In [ ]:
helper_calibration.call_voltage_bowl_calibration(variables, det_diam, calibration_mode)

In [ ]:
variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)
variables.mc_calib_backup = np.copy(variables.mc_calib)

In [ ]:
helper_ion_list.call_ion_list(variables, selector='peak', calibration_mode=calibration_mode)

In [ ]:
helper_mc_plot.call_mc_plot(variables, selector='None')

In [ ]:
variables.data['mc_c (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction Workflow

After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [9]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [ ]:
# import plotly.offline as py
# py.init_notebook_mode(connected=True)

In [10]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected)

Output()

In [ ]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

This tutorial outlines a comprehensive workflow for ion selection and organization. Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both figures and data in CSV and HDF5 formats.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

In [ ]:
helper_ion_selection.call_ion_selection(variables)

In [ ]:
variables.range_data

In [ ]:
variables.range_data.dtypes

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [ ]:
# save the new data
name_save_file = variables.result_data_path + '/' + 'range_' + variables.dataset_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)
# save data in csv format
name_save_file = variables.result_data_path + '/' + 'range_' + variables.dataset_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [ ]:
interact_manual(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [ ]:
helper_visualization.call_visualization(variables)

In [14]:
variables.x

array([16.24153407,  0.64999749,  8.95553548, ..., 34.29849115,
       27.44979228,  4.87282548])

In [ ]:
from pyccapt.calibration.reconstructions import reconstruction

display(reconstruction.reconstruction_plot(variables, element_percentage=[0.001, 0.001, 0.001], opacity=0.5, rotary_fig_save=False, figname='22', save=True, selected_area_specially=False,
                        selected_area_temporally=False, ions_individually_plots=False))